# 本脚本用于部署CSDC gptq int4 量化过的Baichuan-7b-chat 
https://huggingface.co/csdc-atl/Baichuan2-7B-Chat-Int4

### 1. 安装HuggingFace 并下载模型到本地

In [6]:
!pip install huggingface-hub -Uqq 
!pip install -U sagemaker

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_bc2_7b_int4_model")


In [3]:
local_model_path.mkdir(exist_ok=True)
model_name = "csdc-atl/Baichuan2-7B-Chat-Int4"
# commit_hash = "670d17ee403f45334f53121d72feff623cc37de1"

In [4]:
snapshot_download(repo_id=model_name, 
                  # revision=commit_hash,
                  cache_dir=local_model_path)

'LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0'

### 2. 把模型拷贝到S3为后续部署做准备

In [8]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [9]:
s3_model_prefix = "LLM-RAG/workshop/LLM_bc2_7b_int4_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_bc2_7b_int4_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_bc2_7b_int4_deploy_code
model_snapshot_path: LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0


In [10]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_model/.gitattributes
upload: LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_model/config.json
upload: LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0/configuration_baichuan.py to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_model/configuration_baichuan.py
upload: LLM_bc2_7b_int4_model/models--csdc-atl--Baichuan2-7B-Chat-Int4/snapshots/4f5cf556a59556f7ef19456bc389f6c09b0215a0/generation_utils.py to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_model/generation_utils.py
upload: LLM_bc2_7b_int4_model/models--csdc-atl

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [11]:
# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [12]:
!mkdir -p LLM_bc2_7b_int4_deploy_code

In [14]:
%%writefile LLM_bc2_7b_int4_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoModelForCausalLM, AutoTokenizer,set_seed
from transformers.generation import GenerationConfig
STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    tokenizer = AutoTokenizer.from_pretrained(model_location, use_fast=True, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
    model.generation_config = GenerationConfig.from_pretrained(model_location)

    return model, tokenizer


model = None
tokenizer = None
generator = None


def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    model.generation_config.max_new_tokens = max_length
    model.generation_config.top_p = top_p if top_p < 1 else 0.95
    
    messages = construct_message(history,prompt)
    res_generator = model.chat(tokenizer, messages,stream=True)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}
    


def handle(inputs: Input):
    set_seed(1)
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    stream = data.get('stream')
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        messages = construct_message(history,input_sentences)
        response = model.chat(tokenizer, messages,stream=False)
        result = {"outputs": response, "history" : []}
        outputs.add_as_json(result)
        
    return outputs

Overwriting LLM_bc2_7b_int4_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [15]:
%%writefile LLM_bc2_7b_int4_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_model/

Writing LLM_bc2_7b_int4_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [17]:
%%writefile LLM_bc2_7b_int4_deploy_code/requirements.txt
transformers==4.31.0
accelerate
colorama
bitsandbytes
sentencepiece
transformers_stream_generator
cpm_kernels
xformers
auto-gptq

Writing LLM_bc2_7b_int4_deploy_code/requirements.txt


In [18]:
!rm model.tar.gz
!cd LLM_bc2_7b_int4_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_bc2_7b_int4_deploy_code

LLM_bc2_7b_int4_deploy_code/
LLM_bc2_7b_int4_deploy_code/model.py
LLM_bc2_7b_int4_deploy_code/requirements.txt
LLM_bc2_7b_int4_deploy_code/serving.properties


In [19]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_7b_int4_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [20]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"bc2-7b-int4") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)


model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bc2-7b-int4-2023-09-15-01-29-48-581
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/bc2-7b-int4-2023-09-15-01-29-48-581


In [21]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/bc2-7b-int4-2023-09-15-01-29-48-581-config',
 'ResponseMetadata': {'RequestId': '9909bb5f-2ff1-404c-a864-3eec46ceee8b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9909bb5f-2ff1-404c-a864-3eec46ceee8b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Fri, 15 Sep 2023 01:29:56 GMT'},
  'RetryAttempts': 0}}

In [22]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bc2-7b-int4-2023-09-15-01-29-48-581-endpoint


#### 持续检测模型部署进度

In [23]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bc2-7b-int4-2023-09-15-01-29-48-581-endpoint
Status: InService


### 5. 模型测试

In [27]:
!pip install -U boto3 botocore

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/bf/b1/8c254dbb9e315f3b2f5a2a30667a08aed13fd9a00e07028eee701c7ce00d/boto3-1.28.47-py3-none-any.whl.metadata
  Obtaining dependency information for botocore from https://files.pythonhosted.org/packages/16/e7/5b9339e7d076ddacafe51614e1e48100a52ed3ef16465a6cd625e16bde86/botocore-1.31.47-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 28.4 MB/s eta 0:00:00:00:010:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: botocore
    Found existing installation: botocore 1.3

In [24]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.5,
  "top_p":0.9
}

CPU times: user 6.81 ms, sys: 0 ns, total: 6.81 ms
Wall time: 7.36 ms


In [25]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream 

In [26]:
import time


# prompts1 = """what's the weather like there?"""
# history = [
#         [
#             "what is the capital city of china?",
#             "Beijing."
#         ]
#     ]
# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """
# prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
prompts1 = """床前明月光是谁写的"""
history = []
# endpoint_name = 'bc2-13b-stream-2023-09-07-10-59-15-557-endpoint'

start = time.time()
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : history,
                "stream":True
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue
print (f"time:{time.time()-start} s")

EventStreamError: An error occurred (ModelStreamError) when calling the InvokeEndpointWithResponseStream operation: Your model primary did not complete sending the inference response in the allotted time.

## None Stream

In [33]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
# prompts1 = """床前明月光是谁写的"""
# prompts1 = """那里有什么景点"""
# history = [
#         [
#             "中国的首都是哪里",
#             "北京."
#         ]
#     ]
start = time.time()
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

resp = response_model['Body'].read()
end = time.time()-start
print (f"\ntime:{end} s")
# print(resp.decode('utf8'))
print(json.loads(resp)['outputs'])


time:14.78024697303772 s
在遥远的未来，宇宙成为了人类的主要战场。在这个时代，星际帝国之间的战争已经持续了数千年，而这场战争的唯一目标就是控制宇宙中稀缺的资源——能源。

在银河系的边缘，有一个名为“诺瓦”的星球。这个星球上的居民被称为“诺瓦人”，他们拥有高度发达的科技，但他们的星球却因为长期的战争而陷入了困境。为了寻找新的能源，诺瓦星球的领导者决定派遣一支精英舰队前往宇宙深处，寻找传说中的“永恒能源”。

在漫长的宇宙旅行中，诺瓦舰队遇到了一群来自遥远星系的生物——“奥菲利亚人”。奥菲利亚人拥有高度智慧，他们的科技水平远超诺瓦星球。在奥菲利亚人的帮助下，诺瓦舰队成功地找到了永恒能源，但这股强大的能量却给诺瓦舰队带来了巨大的危险。

在永恒能源的影响下，诺瓦舰队的战舰开始发生了异变。原本坚固的战舰逐渐被永恒能源侵蚀，变得脆弱不堪。诺瓦舰队面临着巨大的危机，如果他们不能尽快找到新的能源，整个舰队都将陷入毁灭。

在这个关键时刻，诺瓦舰队的一名年轻指挥官——亚历山大·斯特林站了出来。他提议带领一支小队伍，深入永恒能源的源头，寻找能够解除永恒能源诅咒的方法。尽管这个任务充满了危险，但亚历山大坚信，这是唯一能够拯救诺瓦舰队的方法。

在奥菲利亚人的帮助下，亚历山大和他的队伍成功来到了永恒能源的源头。在这里，他们发现了一个神秘的星球，这个星球上有一种名为“晶石”的能源。晶石的能量与永恒能源截然不同，它既不会侵蚀战舰，也不会带来灾难。

经过研究，亚历山大发现晶石的能量可以与永恒能源相互抵消。他决定将这个秘密带回诺瓦星球，以拯救他的舰队。然而，在这个过程中，亚历山大和他的队伍也遭遇了重重困难


In [38]:

def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

In [52]:
prompts1 = """那里有什么景点"""
history = [
        [
            "中国的首都是哪里",
            "北京."
        ]
    ]

In [53]:
messages=construct_message(history,prompts1)

In [54]:
def _parse_messages(messages, split_role="user"):
    system, rounds = "", []
    round = []
    for i, message in enumerate(messages):
        if message["role"] == "system":
            assert i == 0
            system = message["content"]
            continue
        if message["role"] == split_role and round:
            rounds.append(round)
            round = []
        round.append(message)
    if round:
        rounds.append(round)
    return system, rounds

In [81]:
system, rounds = _parse_messages(messages)
history_tokens = []
for round in rounds[::-1]:
    round_tokens = []
    
    for message in round:
        if message["role"] == "user":
            round_tokens.append(1)
        else:
            round_tokens.append(2)
        round_tokens.extend(message["content"])
    history_tokens = round_tokens + history_tokens  # concat left
print(history_tokens)

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.', 1, '那', '里', '有', '什', '么', '景', '点']


In [73]:
round_tokens

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.']

In [74]:
rounds[::-1]

[[{'role': 'user', 'content': '那里有什么景点'}],
 [{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}]]

In [67]:
rounds

[[{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}],
 [{'role': 'user', 'content': '那里有什么景点'}]]

#### 清除模型Endpoint和config

In [43]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [44]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [45]:
!aws sagemaker delete-model --model-name {model_name}